In [ ]:
import plotly.graph_objects as go
import math

# Define vertices for a regular tetrahedron
A = (0, 0, 0)
B = (1, 0, 0)
C = (0.5, math.sqrt(3)/2, 0)
D = (0.5, math.sqrt(3)/6, math.sqrt(6)/3)

# List all edges as pairs of vertices
edges = [
    (A, B),
    (A, C),
    (A, D),
    (B, C),
    (B, D),
    (C, D)
]

# Prepare lists to hold the coordinates for all edges.
# We use None as a break between each edge so Plotly doesn't connect them.
x_edges, y_edges, z_edges = [], [], []
for edge in edges:
    (x1, y1, z1), (x2, y2, z2) = edge
    x_edges.extend([x1, x2, None])
    y_edges.extend([y1, y2, None])
    z_edges.extend([z1, z2, None])

# Create a 3D scatter plot using lines
fig = go.Figure(data=go.Scatter3d(
    x=x_edges,
    y=y_edges,
    z=z_edges,
    mode='lines',
    line=dict(color='cyan', width=5)
))

# Update layout for a dark grey background with no visible axes
fig.update_layout(
    title="Borders of a Standard 3-Simplex (Tetrahedron)",
    paper_bgcolor='#2c2c2c',
    plot_bgcolor='#2c2c2c',
    font_color='#e0e0e0',
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
    )
)


fig.write_html("my-website/static/simplex.html")


In [2]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width', color='species')
fig.show()